# Creating a simple chatbot using the ChatGPT model

*__NOTE:__
For instructions on running the Jupyter Notebook that contains the labs see instructions here: <https://github.com/retaildevcrews/OpenAI-Labs>*

This sample notebook demonstrates  creation of a simple chatbot that uses the ChatGPT model using the Chat Completion API.  The API takes in an array of messages that have the following parts:
- role (system - sets the overall behavior for the assistant , user - provides specific instructions for the assistant, assistant - stores previous responses and can be used for fine tuning to achieve desired behavior)
- content

For this lab the system message will be a simple description, and the messages sent to the api will be a collection that inclues all of the previous messages and responses.  In addition this example will track the number of tokens in the message to ensure it does not exceed the maximum.

The chatbot labs are based on examples found in the <https://github.com/Azure/openai-samples> repository.


In [1]:
from ffmodel.core.inference_endpoint import InferenceEndpoint

environment_config_path = "~/.chatbot.env"

#solution_config_path = "../solution/configs/00_baseline.yaml"
#solution_config_path = "../solution/configs/01_with_static_context_01.yaml"
#solution_config_path = "../solution/configs/02_with_fewshots_01 _min.yaml"
#solution_config_path = "../solution/configs/03_with_fewshots_01.yaml"
solution_config_path = "../solution/configs/05_with_fewshots_02_min.yaml"
#solution_config_path = "../solution/configs/04_with_fewshots_02.yaml"

endpoint = InferenceEndpoint(solution_config_path, environment_config_path)

2023-10-15 09:26:00,830 name=ffmodel.core.inference_endpoint level=METRICS solution_config 


Loading local environment configs from file.
Loaded 3 configs from file '/Users/jorgeluna/.product-search.env'.
Key vault url not set. Skipping key vault client initialization.
FFModelLogger: Logging will print to console as no AppInsight connection was provided (config `ApplicationInsightConnectionString)`.


[nltk_data] Downloading package punkt to /Users/jorgeluna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 1.0 Define Request to OpenAI API
For these labs we are using the Azure OpenAI Chat Completion API, this is different than the Completions API.  Information about its usage can be found here:
<https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions>
The chat completions API takes a collection of messages to provide history and context of chat to the model.  This function takes a collection of messages which are objects that define a role and content.

We also create a function that calculates the number of tokens received from the message collection so that we can keep track of message size and not exceed threshhold.


In [2]:
import tiktoken


# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(chat_request):
    response = endpoint.execute(chat_request)
    return response.model_output.completions[0]



## 2.0 The conversation

In [3]:
import json
import typing
import uuid

from pydantic import BaseModel
from fastapi.encoders import jsonable_encoder

class CompletionPair(BaseModel):
    user_nl:str
    completion:object

class UserSession(BaseModel):
    session_id:str
    prior_responses:list[CompletionPair]
    sequence: typing.Optional[int]


class ChatRequest(BaseModel):
    user_nl:str
    session:UserSession

# This is the first user message that will be sent to the model. Feel free to update this.
max_response_tokens = 500
token_limit=2048
name = input('Enter Your Name: ')
print ('Welcome to pizzaai.  What can I get for you?')
conversation=[]
session_id=str(uuid.uuid4())
message_count=0
while True:
    request = input(name+':')
    current_message={request}
    chat_request=ChatRequest(user_nl=request,session=UserSession(session_id=session_id,sequence=message_count,prior_responses=conversation))
    if request=="Bye" or request=='bye':
        print('Bot: Bye')
        break
    else:
        payload=json.dumps(jsonable_encoder(chat_request))
        response = send_message(payload)
        interaction = CompletionPair(user_nl=request,completion=response)
        conversation.append(interaction)
        message_count+=1
        print('Bot: ', response)

Welcome to pizzaai.  What can I get for you?


2023-10-15 09:26:24,134 name=ffmodel.core.inference_endpoint level=METRICS final_data_model 
2023-10-15 09:26:24,135 name=ffmodel.core.inference_endpoint level=METRICS request_complete 


Bot:  I'm sorry, but we do not offer candy at our pizza and wings restaurant. We specialize in pizza, wings, and drinks. Is there anything else I can help you with?


2023-10-15 09:26:52,478 name=ffmodel.core.inference_endpoint level=METRICS final_data_model 
2023-10-15 09:26:52,479 name=ffmodel.core.inference_endpoint level=METRICS request_complete 


Bot:  I'm sorry, but we do not offer pie at our pizza and wings restaurant. We specialize in pizza, wings, and drinks. Is there anything else I can help you with?


2023-10-15 09:27:46,684 name=ffmodel.core.inference_endpoint level=METRICS final_data_model 
2023-10-15 09:27:46,685 name=ffmodel.core.inference_endpoint level=METRICS request_complete 


Bot:  I'm sorry, but I am unable to process that request as it is not a valid order for pizza, wings, or drinks. Please let me know if you would like to place an order.


2023-10-15 09:28:15,279 name=ffmodel.core.inference_endpoint level=METRICS final_data_model 
2023-10-15 09:28:15,280 name=ffmodel.core.inference_endpoint level=METRICS request_complete 


Bot:  {"items": [{"type": "drink", "name": "Sprite", "quantity": 2, "size": "2 liter"}, {"type": "custom pizza", "size": "large", "crust": "regular", "quantity": 2, "toppings": ["pepperoni"]}]}
